<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EB%A6%AC%EB%B7%B0/%ED%97%88%EA%B9%85%ED%8E%98%EC%9D%B4%EC%8A%A4%ED%99%9C%EC%9A%A9_roberta_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import tqdm
import pandas as pd
import torch
import datasets

from glob import glob
from tqdm import tqdm

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [ ]:
pip install transformers

In [ ]:
pip install datasets

In [41]:
MODEL_NAME= "klue/roberta-large"# "klue/bert-base", "klue/bert-large", "klue/roberta-base"
batch_size=64
learning_rate=5e-5
num_epochs=1

## 데이터셋 로드

In [42]:
raw_train=load_dataset('csv',data_files='/content/train_aug.csv')
raw_test=load_dataset('csv',data_files='/content/test_preprocessed.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
train,valid=raw_train['train'].train_test_split(test_size=0.1).values()
review_dataset=datasets.DatasetDict({'train':train,'valid':valid,'test':raw_test['train']})
review_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'reviews', 'target'],
        num_rows: 22500
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'id', 'reviews', 'target'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id', 'reviews'],
        num_rows: 25000
    })
})

In [44]:
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
print(tokenizer.tokenize(train['reviews'][0]))

['가격', '값', '##하', '##는', '것', '같', '##아', '##요']


In [45]:
def tokenize_function(example):
  return tokenizer(example["reviews"],truncation=True)

tokenized_datasets=review_dataset.map(tokenize_function,batched=True)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [46]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'reviews', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 22500
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'id', 'reviews', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id', 'reviews', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
})

In [47]:
tokenized_datasets=tokenized_datasets.remove_columns(['id','reviews','Unnamed: 0'])
tokenized_datasets['train']=tokenized_datasets['train'].rename_column("target","labels")
tokenized_datasets['valid']=tokenized_datasets['valid'].rename_column("target",'labels')

tokenized_datasets.set_format("torch")#여기서 왜 format을 torch로 설정해주는 것인가. 
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [48]:
from torch.utils.data import DataLoader

train_dataloader=DataLoader(tokenized_datasets["train"],shuffle=True,batch_size=batch_size,collate_fn=data_collator)
valid_dataloader=DataLoader(tokenized_datasets["valid"],shuffle=True,batch_size=batch_size,collate_fn=data_collator)
eval_datalaoder=DataLoader(tokenized_datasets["test"],shuffle=False,batch_size=batch_size,collate_fn=data_collator)

In [49]:
for batch in train_dataloader:
  break

{k:v.shape for k,v in batch.items()}

{'attention_mask': torch.Size([64, 73]),
 'input_ids': torch.Size([64, 73]),
 'labels': torch.Size([64]),
 'token_type_ids': torch.Size([64, 73])}

## 모델 로드

In [50]:
model=AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=6)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classif

In [51]:
from transformers import get_scheduler, AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

352


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [52]:
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [53]:
from tqdm.auto import tqdm

progress_bar=tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch={k:v.to(device) for k,v in batch.items()}
    outputs=model(**batch)
    loss=outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  model.save_pretrained(f"./result/{MODEL_NAME}/{epoch}")
  tokenizer.save_pretrained(f"./result/{MODEL_NAME}/{epoch}")

  0%|          | 0/352 [00:00<?, ?it/s]

RuntimeError: ignored